# embedding

    https://blog.csdn.net/pipisorry/article/details/76095118?utm_source=blogxgwz0
    
    https://blog.csdn.net/mylove0414/article/details/61616617?utm_source=blogxgwz1
    
    https://baijiahao.baidu.com/s?id=1602069319915188130&wfr=spider&for=pc
    
    https://blog.csdn.net/appleml/article/details/78595724 pytorch加载已训练好的word-embedding
    
    那么我们为什么要使用嵌入层 Embedding呢? 主要有这两大原因:

    1、使用One-hot 方法编码的向量会很高维也很稀疏。假设我们在做自然语言处理（NLP）中遇到了一个包含2000个词的字典，当时用One-hot编码时，每一个词会被一个包含2000个整数的向量来表示，其中1999个数字是0，要是我的字典再大一点的话这种方法的计算效率岂不是大打折扣？

    2、训练神经网络的过程中，每个嵌入的向量都会得到更新。如果你看到了博客上面的图片你就会发现在多维空间中词与词之间有多少相似性，这使我们能可视化的了解词语之间的关系，不仅仅是词语，任何能通过嵌入层 Embedding 转换成向量的内容都可以这样做。
    
    
    embedding 算法主要用于处理稀疏特征，应用于NLP、推荐、广告等领域。所以word2vec 只是embbeding 思想的一个应用，而不是全部。
    
    常见的特征降维方法主要有PCA、SVD等。而embedding的主要目的也是对（稀疏）特征进行降维，它降维的方式可以类比为一个全连接层（没有激活函数），通过 embedding 层的权重矩阵计算来降低维度。

In [29]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torch.nn.functional as F
# import torch.Value.Variable as Variable
# https://my.oschina.net/earnp/blog/1113897  使用word embedding做自然语言处理的词语预测
# 数据预处理 首先我们给出了一段文章作为我们的训练集

CONTEXT_SIZE = 2  # 表示我们想由前面的几个单词来预测这个单词，这里设置为2，就是说我们希望通过这个单词的前两个单词来预测这一个单词。
EMBEDDING_DIM = 10
# We will use Shakespeare Sonnet 2
test_sentence = """When forty winters shall besiege thy brow,
And dig deep trenches in thy beauty's field,
Thy youth's proud livery so gazed on now,
Will be a totter'd weed of small worth held:
Then being asked, where all thy beauty lies,
Where all the treasure of thy lusty days;
To say, within thine own deep sunken eyes,
Were an all-eating shame, and thriftless praise.
How much more praise deserv'd thy beauty's use,
If thou couldst answer 'This fair child of mine
Shall sum my count, and make my old excuse,'
Proving his beauty by succession thine!
This were to be new made when thou art old,
And see thy blood warm when thou feel'st it cold.""".split()

trigram = [((test_sentence[i], test_sentence[i+1]), test_sentence[i+2])
           for i in range(len(test_sentence)-2)]

vocb = set(test_sentence) # 通过set将重复的单词去掉
word_to_idx = {word: i for i, word in enumerate(vocb)}
idx_to_word = {word_to_idx[word]: word for word in word_to_idx}

# 定义模型
class NgramModel(nn.Module):
    def __init__(self, vocb_size, context_size, n_dim):
        super(NgramModel, self).__init__()
        self.n_word = vocb_size
        # 注意Embedding初始化后， 不参与回归，每次的词的词向量是相同的。
        self.embedding = nn.Embedding(self.n_word, n_dim)
        self.linear1 = nn.Linear(context_size*n_dim, 128)
        self.linear2 = nn.Linear(128, self.n_word)

    def forward(self, x):
        emb = self.embedding(x)
        emb = emb.view(1, -1)
        out = self.linear1(emb)
        out = F.relu(out)
        out = self.linear2(out)
#         print(out)
        log_prob = F.log_softmax(out)
#         print(log_prob)
        return log_prob

ngrammodel = NgramModel(len(word_to_idx), CONTEXT_SIZE, 100)
criterion = nn.NLLLoss()
optimizer = optim.SGD(ngrammodel.parameters(), lr=1e-3)

for epoch in range(10):
    print('epoch: {}'.format(epoch+1))
    print('*'*10)
    running_loss = 0
    for data in trigram:
        word, label = data
        word = Variable(torch.LongTensor([word_to_idx[i] for i in word]))
        label = Variable(torch.LongTensor([word_to_idx[label]]))
        # forward
        out = ngrammodel(word)
        loss = criterion(out, label)
        running_loss += loss.data[0]
        # backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print('Loss: {:.6f}'.format(running_loss / len(word_to_idx)))
    
word, label = trigram[30]
word = Variable(torch.LongTensor([word_to_idx[i] for i in word]))
out = ngrammodel(word)
_, predict_label = torch.max(out, 1)
print(predict_label)
predict_word = idx_to_word[predict_label.item()]
print('real word is {}, predict word is {}'.format(label, predict_word))    


epoch: 1
**********


/home/hecong/venv/lib/python3.6/site-packages/ipykernel_launcher.py:52: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
/home/hecong/venv/lib/python3.6/site-packages/ipykernel_launcher.py:71: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


Loss: 5.348613
epoch: 2
**********
Loss: 5.287601
epoch: 3
**********
Loss: 5.227542
epoch: 4
**********
Loss: 5.168027
epoch: 5
**********
Loss: 5.108868
epoch: 6
**********
Loss: 5.050111
epoch: 7
**********
Loss: 4.991744
epoch: 8
**********
Loss: 4.933736
epoch: 9
**********
Loss: 4.875984
epoch: 10
**********
Loss: 4.818320
tensor([44])
real word is Then, predict word is Then


In [26]:
print(predict_label.item())

27


# Log-Sigmoid

    https://baike.baidu.com/item/S%E5%9E%8B%E5%87%BD%E6%95%B0/19178062
    
    https://blog.csdn.net/zhangxb35/article/details/72464152?utm_source=itdadao&utm_medium=referral pytorch loss function 总结
    
    http://yangli.name/2017/12/25/20171225pytorch2/  pytorch:激活函数、损失函数小结
    
    S型函数（Sigmoid function）是BP神经网络中常用的非线性作用函数，即sigmoid函数，公式是f(x)=1/(1+e^-x)（-x是幂数）。Sigmoid函数又分为Log-Sigmoid函数和Tan-Sigmoid函数。
    由于BP神经网络的传递函数必须可微，所以感知器的传递函数--二值函数在这里不可用，故BP神经网络一般使用Sigmoid函数或者线性函数作为传递函数。而Sigmoid函数又分为Log-Sigmoid函数（一般所说的S型函数就是这个的简称）和Tan-Sigmoid函数（又称为双曲正切S型函数），前者的值域为（0，1），后者的值域为（-1,1）。
    
    m = nn.LogSigmoid()
    input = torch.randn(2)
    print(input)
    output = m(input)
    print(output)    

# cosine

    https://baike.baidu.com/item/%E4%BD%99%E5%BC%A6/73670?fr=aladdin
    
    https://baike.baidu.com/item/%E4%BD%99%E5%BC%A6%E7%9B%B8%E4%BC%BC%E5%BA%A6/17509249
    
    http://www.ruanyifeng.com/blog/2013/03/tf-idf.html TF-IDF与余弦相似性的应用（一）：自动提取关键词
    
    http://www.ruanyifeng.com/blog/2013/03/cosine_similarity.html TF-IDF与余弦相似性的应用（二）：找出相似文章
    
    http://www.ruanyifeng.com/blog/2013/03/automatic_summarization.html TF-IDF与余弦相似性的应用（三）：自动摘要
    
    https://www.2cto.com/kf/201612/579595.html 利用余弦计算文本相似性
    
    https://en.wikipedia.org/wiki/Cosine_similarity
    
    余弦定理亦称第二余弦定理。关于三角形边角关系的重要定理之一。该定理断言：三角形任一边的平方等于其他两边平方和减去这两边与它们夹角的余弦的积的两倍。若a、b、c分别表示∆ABC中A、B、C的对边，则余弦定理可表述为 [1]  ：



    
    角 的邻边比斜边叫做 的余弦，记作（由余弦英文cosine简写 ），即角的邻边/斜边（直角三角形）。记作cos=x/r。
    
    余弦相似度，又称为余弦相似性，是通过计算两个向量的夹角余弦值来评估他们的相似度。余弦相似度将向量根据坐标值，绘制到向量空间中，如最常见的二维空间。
    
    余弦相似性通过测量两个向量的夹角的余弦值来度量它们之间的相似性。0度角的余弦值是1，而其他任何角度的余弦值都不大于1；并且其最小值是-1。从而两个向量之间的角度的余弦值确定两个向量是否大致指向相同的方向。两个向量有相同的指向时，余弦相似度的值为1；两个向量夹角为90°时，余弦相似度的值为0；两个向量指向完全相反的方向时，余弦相似度的值为-1。这结果是与向量的长度无关的，仅仅与向量的指向方向相关。余弦相似度通常用于正空间，因此给出的值为0到1之间。
    注意这上下界对任何维度的向量空间中都适用，而且余弦相似性最常用于高维正空间。例如在信息检索中，每个词项被赋予不同的维度，而一个文档由一个向量表示，其各个维度上的值对应于该词项在文档中出现的频率。余弦相似度因此可以给出两篇文档在其主题方面的相似度。
    
    